In [1]:
import pandas as pd
import pixiedust

import requests
import json

Pixiedust database opened successfully


In [2]:
df = pd.read_csv('DB_ready.csv')

In [3]:
config = dict(
    username="2660e8fa-aac2-45ef-81ba-217967728ad2-bluemix", 
    password="9cd1b00e6b6927e663bfa0e2e9d677e163378a7c98e09bfb598dd74a4439fc7a",
    database="cfc_patient_db")
params = {'include_docs': 'true'}

In [4]:
url = "https://{username}:{password}@{username}.cloudant.com/{database}/_all_docs".format(**config)

In [5]:
auth_headers = {"Content-Type": "application/x-www-form-urlencoded"}
r = requests.get(url=url, params=params)
data = json.loads(r.text)

In [6]:
print(json.dumps(data, indent=2))

{
  "total_rows": 1125,
  "offset": 0,
  "rows": [
    {
      "id": "88b2a4331fa1b0d9be1594ac6c000bd3",
      "key": "88b2a4331fa1b0d9be1594ac6c000bd3",
      "value": {
        "rev": "2-ecca81292e8562dda6b3bcd641dbe7e2"
      },
      "doc": {
        "_id": "88b2a4331fa1b0d9be1594ac6c000bd3",
        "_rev": "2-ecca81292e8562dda6b3bcd641dbe7e2",
        "": "907",
        "Items": "Plastic rolls (UNCHR)",
        "Origin": "UNHCR",
        "Items grouped": "Plastic rolls (UNCHR)",
        "Item group filter": "",
        "Category": "Shelter",
        "Unit": "unit",
        "amount": "1.0",
        "Mission": "MISSION 17/5 *01",
        "District": "Gorkha",
        "VDC": "Chhoprak",
        "Village/Ward/extra info": "Gaira, Jaisithok, Phirphire, Arubot, ward 8",
        "lat": "28.0556",
        "lon": "84.5881",
        "District Latitude": "28.33333",
        "District Longitude": "84.83333",
        "Date": "2015-05-17",
        "Location name": "Chhoprak, Gaira, Jaisithok, 

In [7]:
df = [row["doc"] for row in data["rows"]]

In [8]:
df =pd.DataFrame(df)

In [9]:
df[["lat","lon","amount"]] = df[["lat","lon","amount"]].apply(pd.to_numeric)

In [13]:
#df[["amount"]] = df[["amount"]].astype(int)

In [11]:
display(df)

In [91]:
# creating patient Data
class Person:
    def __init__(self, name, supply_name, supply_amount, location):
        self.name = name
        self.supply_name = supply_name
        self.supply_amount = supply_amount
        self.location = location
        
class Aid_Station:
    def __init__(self, name, items, lat, lon):
        self.name = name
        self.items = items
        self.lat = lat
        self.lon = lon
    

import random
items = set([doc["Items"] for doc in [row["doc"] for row in data["rows"]] if "Items" in doc])
locations = dict([(doc["Location name"], (doc["lat"],doc["lon"])) for doc in [row["doc"] for row in data["rows"]] if "Items" in doc])
aid_stations = []

for name, loc in locations.items():
    aid_stations.append(Aid_Station(name,{},loc[0],loc[1]))

people = []
people_names = open("people.txt")
for name in people_names:
    people.append(Person(name, random.choice(list(items)), random.randint(0,100), random.choice(list(locations.keys()))))


In [96]:
for person in people:
    if person.location in [station.name for station in aid_stations]:
        if person.supply_name in aid_stations[person.location].items.keys():
            aid_stations[person.location].items[person.supply_name] = \
            aid_stations[person.location].items[person.supply_name] - person.supply_amount 
        else:
            aid_stations[person.location].items[person.supply_name] = person.supply_amount

TypeError: list indices must be integers or slices, not str